In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
using_colab = True
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

    #!mkdir images
    #!wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg

    #!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

PyTorch version: 2.5.1+cu121
Torchvision version: 0.20.1+cu121
CUDA is available: True
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-97tem4q0
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-97tem4q0
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done
  Created wheel for segment_anything: filename=segment_anything-1.0-py3-none-any.whl size=36592 sha256=09775fee848791642dbf832a58bd56f0dbf05cd284ca8bda5a11c5f0b75b1674
  Stored in directory: /tmp/pip-ephem-wheel-cache-ajzery5f/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment_anything


In [3]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
import torch
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator

# Define paths and device
sam_checkpoint = "/content/drive/MyDrive/SAM_model/sam_vit_h_4b8939.pth"
model_type = "vit_h"
device = "cuda"

# Initialize SAM model and mask generator
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
print("done")

/usr/local/lib/python3.10/dist-packages/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


done


In [9]:
# Example usage
file_name = "image_2"
image_path = f"/content/{file_name}.jpg"
image = cv2.imread(image_path)
# height, width, channels = image.shape
# print(image.shape)
# points_points_per_side = int(min(height, width)/100)
points_points_per_side = 3
points_points_per_side

3

In [ ]:
# mask_generator = SamAutomaticMaskGenerator(sam)
# masks = mask_generator.generate(image)

In [14]:
mask_generator_3 = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=2,
    pred_iou_thresh=0.60,
    stability_score_thresh=0.60,
    crop_n_layers=1,
    crop_n_points_downscale_factor=2,
    #min_mask_region_area=100000,
)

In [15]:
import os
import cv2
import numpy as np

def update_masks(masks, image):
    """
    Function to update masks by ensuring non-overlapping segmentation regions.

    Args:
    - masks: List of dictionaries, each containing a 'segmentation' mask.
    - image: Input image (used for identifying unsegmented regions).

    Returns:
    - updated_masks: List of updated masks with unique segmentation regions, including the unsegmented region.
    """
    used_regions = np.zeros_like(masks[0]['segmentation'], dtype=bool)  # Initialize an empty region mask
    updated_masks = []

    for idx, mask in enumerate(masks):
        # Extract the segmentation mask
        segmentation_mask = mask['segmentation']

        # Ensure non-overlapping by subtracting already used regions
        unique_mask = np.logical_and(segmentation_mask, ~used_regions)

        # Update used regions to include the current mask
        used_regions = np.logical_or(used_regions, unique_mask)

        # Append the updated mask with unique segmentation region
        updated_masks.append({'segmentation': unique_mask})

    # Add unsegmented regions as a new mask
    unsegmented_regions = ~used_regions
    if np.any(unsegmented_regions):
        updated_masks.append({'segmentation': unsegmented_regions})

    return updated_masks


# def update_masks(masks, image):
#     """
#     Function to update masks by ensuring non-overlapping segmentation regions.

#     Args:
#     - masks: List of dictionaries, each containing a 'segmentation' mask.
#     - image: Input image (used for saving unsegmented regions).

#     Returns:
#     - updated_masks: List of updated masks with unique segmentation regions.
#     """
#     used_regions = np.zeros_like(masks[0]['segmentation'], dtype=bool)  # Initialize an empty region mask
#     updated_masks = []

#     for idx, mask in enumerate(masks):
#         # Extract the segmentation mask
#         segmentation_mask = mask['segmentation']

#         # Ensure non-overlapping by subtracting already used regions
#         unique_mask = np.logical_and(segmentation_mask, ~used_regions)

#         # Update used regions to include the current mask
#         used_regions = np.logical_or(used_regions, unique_mask)

#         # Append the updated mask with unique segmentation region
#         updated_masks.append({'segmentation': unique_mask})
#     unseg_id = len(update_masks)+1
#     unsegmented_regions = ~used_regions

#     if np.any(unsegmented_regions):
#         # Create a blank RGBA image for unsegmented regions
#         unsegmented_image = np.zeros((image.shape[0], image.shape[1], 4), dtype=np.uint8)

#         # Copy the RGB values of the unsegmented regions
#         unsegmented_image[unsegmented_regions, :3] = image[unsegmented_regions]

#         # Set alpha channel (255 for the unsegmented region, 0 for background)
#         unsegmented_image[unsegmented_regions, 3] = 255

#         # Save the unsegmented region image
#         unsegmented_filename = os.path.join(output_dir, f"{file_name}_segment_{unseg_id}.png")
#         cv2.imwrite(unsegmented_filename, cv2.cvtColor(unsegmented_image, cv2.COLOR_RGBA2BGRA))
#         print(f"Saved unsegmented regions: {unsegmented_filename}")

#     return updated_masks


def save_mask_objects(image, masks, output_dir, file_name):
    """
    Function to save masks as transparent PNG images.

    Args:
    - image: Input image for the segmentation.
    - masks: List of masks to be saved.
    - output_dir: Directory to save the output images.
    - file_name: Base name for the output files.
    """
    for idx, mask in enumerate(masks):
        if not np.any(mask['segmentation']):
            continue

        # Create a blank RGBA image (adds alpha channel)
        transparent_image = np.zeros((image.shape[0], image.shape[1], 4), dtype=np.uint8)

        # Copy the segment's RGB values to the new image
        transparent_image[mask['segmentation'], :3] = image[mask['segmentation']]

        # Set alpha channel (255 for the mask region, 0 for background)
        transparent_image[mask['segmentation'], 3] = 255

        # Save the segmented image
        segment_filename = os.path.join(output_dir, f"{file_name}_segment_{idx}.png")
        cv2.imwrite(segment_filename, cv2.cvtColor(transparent_image, cv2.COLOR_RGBA2BGRA))
        print(f"Saved: {segment_filename}")


# Example usage
# file_name = "Van_Skeleton"
# image_path = f"/content/{file_name}.png"
# image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
output_dir = f"/content/drive/MyDrive/Seg_output/{file_name}"
import shutil
# Check if the directory exists
if os.path.exists(output_dir):
    # Remove the directory and its contents
    shutil.rmtree(output_dir)

# Recreate the directory
#os.makedirs(output_dir, exist_ok=True)
# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Assuming mask_generator_3 is defined elsewhere
masks2 = mask_generator_3.generate(image_rgb)
#masks2 = mask_generator.generate(image_rgb)
masks2 = sorted(masks2, key=lambda mask: np.count_nonzero(mask['segmentation']), reverse=False)
nop_masks = update_masks(masks2, image_rgb)
sorted_masks = sorted(nop_masks, key=lambda mask: np.count_nonzero(mask['segmentation']), reverse=True)

# Save non-overlapping masks with an optional area threshold
save_mask_objects(image_rgb, sorted_masks, output_dir, file_name)


Saved: /content/drive/MyDrive/Seg_output/image_2/image_2_segment_0.png
Saved: /content/drive/MyDrive/Seg_output/image_2/image_2_segment_1.png
Saved: /content/drive/MyDrive/Seg_output/image_2/image_2_segment_2.png
Saved: /content/drive/MyDrive/Seg_output/image_2/image_2_segment_3.png
Saved: /content/drive/MyDrive/Seg_output/image_2/image_2_segment_4.png
Saved: /content/drive/MyDrive/Seg_output/image_2/image_2_segment_5.png
Saved: /content/drive/MyDrive/Seg_output/image_2/image_2_segment_6.png
Saved: /content/drive/MyDrive/Seg_output/image_2/image_2_segment_7.png
Saved: /content/drive/MyDrive/Seg_output/image_2/image_2_segment_8.png
Saved: /content/drive/MyDrive/Seg_output/image_2/image_2_segment_9.png
Saved: /content/drive/MyDrive/Seg_output/image_2/image_2_segment_10.png
Saved: /content/drive/MyDrive/Seg_output/image_2/image_2_segment_11.png
Saved: /content/drive/MyDrive/Seg_output/image_2/image_2_segment_12.png
Saved: /content/drive/MyDrive/Seg_output/image_2/image_2_segment_13.png


In [ ]:
import os
import cv2
import numpy as np

def save_mask_objects(image, masks, output_dir, area_threshold):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Initialize an empty array to track used regions
    used_regions = np.zeros(image.shape[:2], dtype=bool)

    # Save each segment as an image with a transparent background
    for idx, mask in enumerate(masks):
        # Extract the segmentation mask
        segmentation_mask = mask['segmentation']

        # Ensure non-overlapping by subtracting already used regions
        unique_mask = np.logical_and(segmentation_mask, ~used_regions)

        # Update used regions to include the current mask
        used_regions = np.logical_or(used_regions, unique_mask)

        # Skip if the mask is empty after removing overlaps
        if not np.any(unique_mask):
            continue

        # Create a blank RGBA image (adds alpha channel)
        transparent_image = np.zeros((image.shape[0], image.shape[1], 4), dtype=np.uint8)

        # Copy the segment's RGB values to the new image
        transparent_image[unique_mask, :3] = image[unique_mask]

        # Set alpha channel (255 for the mask region, 0 for background)
        transparent_image[unique_mask, 3] = 255

        # Save the segmented image
        segment_filename = os.path.join(output_dir, f"segment_{idx+1}.png")
        cv2.imwrite(segment_filename, cv2.cvtColor(transparent_image, cv2.COLOR_RGBA2BGRA))
        print(f"Saved: {segment_filename}")

    # Handle unsegmented regions
    unsegmented_regions = ~used_regions

    if np.any(unsegmented_regions):
        # Create a blank RGBA image for unsegmented regions
        unsegmented_image = np.zeros((image.shape[0], image.shape[1], 4), dtype=np.uint8)

        # Copy the RGB values of the unsegmented regions
        unsegmented_image[unsegmented_regions, :3] = image[unsegmented_regions]

        # Set alpha channel (255 for the unsegmented region, 0 for background)
        unsegmented_image[unsegmented_regions, 3] = 255

        # Save the unsegmented region image
        unsegmented_filename = os.path.join(output_dir, "segment_0.png")
        cv2.imwrite(unsegmented_filename, cv2.cvtColor(unsegmented_image, cv2.COLOR_RGBA2BGRA))
        print(f"Saved unsegmented regions: {unsegmented_filename}")

# Example usage
file_name="pavan"
image_path = f"/content/{file_name}.jpg"
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
output_dir = f"/content/drive/MyDrive/Seg_output/{file_name}"
os.makedirs(output_dir, exist_ok=True)
masks2 = mask_generator_3.generate(image)  # Assuming this is defined elsewhere

# Save non-overlapping masks and unsegmented regions
save_mask_objects(image, masks2, output_dir, area_threshold=0)


In [ ]:
import os
import cv2
import numpy as np

def save_mask_objects(image, masks, output_dir, area_threshold):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Initialize an empty array to track used regions
    used_regions = np.zeros(image.shape[:2], dtype=bool)

    # List to store segments and their areas for sorting
    segments_with_area = []

    # Save each segment as an image with a transparent background
    for idx, mask in enumerate(masks):
        # Extract the segmentation mask
        segmentation_mask = mask['segmentation']

        # Ensure non-overlapping by subtracting already used regions
        unique_mask = np.logical_and(segmentation_mask, ~used_regions)

        # Update used regions to include the current mask
        used_regions = np.logical_or(used_regions, unique_mask)

        # Skip if the mask is empty after removing overlaps
        if not np.any(unique_mask):
            continue

        # Calculate the area of the segment (number of pixels in the mask)
        area = np.count_nonzero(unique_mask)

        # Print the area of the current segment
        print(f"Segment {idx+1} area: {area}")

        # Add the segment and its area to the list
        segments_with_area.append((idx, unique_mask, area))

    # Sort the segments by area (in descending order)
    segments_with_area.sort(key=lambda x: x[2], reverse=True)

    # Process and save each segment, sorted by area
    for idx2, unique_mask, area in segments_with_area:
        # Create a blank RGBA image (adds alpha channel)
        transparent_image = np.zeros((image.shape[0], image.shape[1], 4), dtype=np.uint8)

        area = np.count_nonzero(unique_mask)
        print(idx2, area)

        # Copy the segment's RGB values to the new image
        transparent_image[unique_mask, :3] = image[unique_mask]

        # Set alpha channel (255 for the mask region, 0 for background)
        transparent_image[unique_mask, 3] = 255

        # Generate a unique filename based on segment index and area
        segment_filename = os.path.join(output_dir, f"{file_name}_segment_{idx+1}_area_{area}.png")

        # Save the segmented image
        cv2.imwrite(segment_filename, cv2.cvtColor(transparent_image, cv2.COLOR_RGBA2BGRA))
        print(f"Saved: {segment_filename}")

    # Handle unsegmented regions
    unsegmented_regions = ~used_regions

    if np.any(unsegmented_regions):
        # Create a blank RGBA image for unsegmented regions
        unsegmented_image = np.zeros((image.shape[0], image.shape[1], 4), dtype=np.uint8)

        # Copy the RGB values of the unsegmented regions
        unsegmented_image[unsegmented_regions, :3] = image[unsegmented_regions]

        # Set alpha channel (255 for the unsegmented region, 0 for background)
        unsegmented_image[unsegmented_regions, 3] = 255

        # Save the unsegmented region image with a unique filename
        unsegmented_filename = os.path.join(output_dir, f"{file_name}_segment_0_unsegmented.png")
        cv2.imwrite(unsegmented_filename, cv2.cvtColor(unsegmented_image, cv2.COLOR_RGBA2BGRA))
        print(f"Saved unsegmented regions: {unsegmented_filename}")

# Example usage
file_name = "pavan"
image_path = f"/content/{file_name}.jpg"
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
output_dir = f"/content/drive/MyDrive/Seg_output/{file_name}"
os.makedirs(output_dir, exist_ok=True)
masks2 = mask_generator_3.generate(image)  # Assuming this is defined elsewhere

# Save non-overlapping masks and unsegmented regions
save_mask_objects(image, masks2, output_dir, area_threshold=0)


Segment 1 area: 358789
Segment 2 area: 1919876
Segment 3 area: 2281325
Segment 4 area: 8012
Segment 5 area: 1173189
2 2281325
Saved: /content/drive/MyDrive/Seg_output/pavan/pavan_segment_5_area_2281325.png
1 1919876
Saved: /content/drive/MyDrive/Seg_output/pavan/pavan_segment_5_area_1919876.png
4 1173189
Saved: /content/drive/MyDrive/Seg_output/pavan/pavan_segment_5_area_1173189.png
0 358789
Saved: /content/drive/MyDrive/Seg_output/pavan/pavan_segment_5_area_358789.png
3 8012
Saved: /content/drive/MyDrive/Seg_output/pavan/pavan_segment_5_area_8012.png
Saved unsegmented regions: /content/drive/MyDrive/Seg_output/pavan/pavan_segment_0_unsegmented.png


In [ ]:
import os
import cv2
import numpy as np

def save_mask_objects(image, masks, output_dir, area_threshold):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Initialize an empty array to track used regions
    used_regions = np.zeros(image.shape[:2], dtype=bool)

    # List to store segments and their areas for sorting
    segments_with_area = []

    # Save each segment as an image with a transparent background
    for idx, mask in enumerate(masks):
        # Extract the segmentation mask
        segmentation_mask = mask['segmentation']

        # Ensure non-overlapping by subtracting already used regions
        unique_mask = np.logical_and(segmentation_mask, ~used_regions)

        # Update used regions to include the current mask
        used_regions = np.logical_or(used_regions, unique_mask)

        # Skip if the mask is empty after removing overlaps
        if not np.any(unique_mask):
            continue

        # Calculate the area of the segment (number of pixels in the mask)
        area = np.count_nonzero(unique_mask)

        # Add the segment and its area to the list
        segments_with_area.append((idx, unique_mask, area))

    # Sort the segments by area (in descending order)
    segments_with_area.sort(key=lambda x: x[2], reverse=True)

    # Process and save each segment, sorted by area
    for idx, unique_mask, area in segments_with_area:
        # Create a blank RGBA image (adds alpha channel)
        transparent_image = np.zeros((image.shape[0], image.shape[1], 4), dtype=np.uint8)

        # Copy the segment's RGB values to the new image
        transparent_image[unique_mask, :3] = image[unique_mask]

        # Set alpha channel (255 for the mask region, 0 for background)
        transparent_image[unique_mask, 3] = 255

        # Save the segmented image
        segment_filename = os.path.join(output_dir, f"{file_name}segment_{idx+1}.png")
        cv2.imwrite(segment_filename, cv2.cvtColor(transparent_image, cv2.COLOR_RGBA2BGRA))
        print(f"Saved: {segment_filename}")

    # Handle unsegmented regions
    unsegmented_regions = ~used_regions

    if np.any(unsegmented_regions):
        # Create a blank RGBA image for unsegmented regions
        unsegmented_image = np.zeros((image.shape[0], image.shape[1], 4), dtype=np.uint8)

        # Copy the RGB values of the unsegmented regions
        unsegmented_image[unsegmented_regions, :3] = image[unsegmented_regions]

        # Set alpha channel (255 for the unsegmented region, 0 for background)
        unsegmented_image[unsegmented_regions, 3] = 255

        # Save the unsegmented region image
        unsegmented_filename = os.path.join(output_dir, f"{file_name}segment_0.png")
        cv2.imwrite(unsegmented_filename, cv2.cvtColor(unsegmented_image, cv2.COLOR_RGBA2BGRA))
        print(f"Saved unsegmented regions: {unsegmented_filename}")

# Example usage
file_name = "pavan"
image_path = f"/content/{file_name}.jpg"
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
output_dir = f"/content/drive/MyDrive/Seg_output/{file_name}"
os.makedirs(output_dir, exist_ok=True)
masks2 = mask_generator_3.generate(image)  # Assuming this is defined elsewhere

# Save non-overlapping masks and unsegmented regions
save_mask_objects(image, masks2, output_dir, area_threshold=0)


Saved: /content/drive/MyDrive/Seg_output/pavan/segment_3.png
Saved: /content/drive/MyDrive/Seg_output/pavan/segment_2.png
Saved: /content/drive/MyDrive/Seg_output/pavan/segment_5.png
Saved: /content/drive/MyDrive/Seg_output/pavan/segment_1.png
Saved: /content/drive/MyDrive/Seg_output/pavan/segment_4.png
Saved unsegmented regions: /content/drive/MyDrive/Seg_output/pavan/segment_0.png


In [ ]:
import os
import cv2
import numpy as np

def save_mask_objects(image, masks, output_dir, area_threshold):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Initialize an empty array to track used regions
    used_regions = np.zeros(image.shape[:2], dtype=bool)

    # Save each segment as an image with a transparent background
    for idx, mask in enumerate(masks):
        # Extract the segmentation mask
        segmentation_mask = mask['segmentation']

        # Ensure non-overlapping by subtracting already used regions
        unique_mask = np.logical_and(segmentation_mask, ~used_regions)

        # Update used regions to include the current mask
        used_regions = np.logical_or(used_regions, unique_mask)

        # Skip if the mask is empty after removing overlaps
        if not np.any(unique_mask):
            continue

        # Create a blank RGBA image (adds alpha channel)
        transparent_image = np.zeros((image.shape[0], image.shape[1], 4), dtype=np.uint8)

        # Copy the segment's RGB values to the new image
        transparent_image[unique_mask, :3] = image[unique_mask]

        # Set alpha channel (255 for the mask region, 0 for background)
        transparent_image[unique_mask, 3] = 255

        # Save the segmented image
        segment_filename = os.path.join(output_dir, f"segment_{idx+1}.png")
        cv2.imwrite(segment_filename, cv2.cvtColor(transparent_image, cv2.COLOR_RGBA2BGRA))
        print(f"Saved: {segment_filename}")

    # Handle unsegmented regions
    unsegmented_regions = ~used_regions

    if np.any(unsegmented_regions):
        # Create a blank RGBA image for unsegmented regions
        unsegmented_image = np.zeros((image.shape[0], image.shape[1], 4), dtype=np.uint8)

        # Copy the RGB values of the unsegmented regions
        unsegmented_image[unsegmented_regions, :3] = image[unsegmented_regions]

        # Set alpha channel (255 for the unsegmented region, 0 for background)
        unsegmented_image[unsegmented_regions, 3] = 255

        # Save the unsegmented region image
        unsegmented_filename = os.path.join(output_dir, "segment_0.png")
        cv2.imwrite(unsegmented_filename, cv2.cvtColor(unsegmented_image, cv2.COLOR_RGBA2BGRA))
        print(f"Saved unsegmented regions: {unsegmented_filename}")

# Example usage
file_name="pavan"
image_path = f"/content/{file_name}.jpg"
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
output_dir = f"/content/drive/MyDrive/Seg_output/{file_name}"
os.makedirs(output_dir, exist_ok=True)
masks2 = mask_generator_3.generate(image)  # Assuming this is defined elsewhere

# Save non-overlapping masks and unsegmented regions
save_mask_objects(image, masks2, output_dir, area_threshold=0)


Saved: /content/drive/MyDrive/Seg_output/pavan/segment_1.png
Saved: /content/drive/MyDrive/Seg_output/pavan/segment_2.png
Saved: /content/drive/MyDrive/Seg_output/pavan/segment_3.png
Saved: /content/drive/MyDrive/Seg_output/pavan/segment_4.png
Saved: /content/drive/MyDrive/Seg_output/pavan/segment_5.png
Saved: /content/drive/MyDrive/Seg_output/pavan/segment_6.png
Saved: /content/drive/MyDrive/Seg_output/pavan/segment_7.png
Saved: /content/drive/MyDrive/Seg_output/pavan/segment_8.png
Saved: /content/drive/MyDrive/Seg_output/pavan/segment_9.png
Saved: /content/drive/MyDrive/Seg_output/pavan/segment_10.png
Saved: /content/drive/MyDrive/Seg_output/pavan/segment_11.png
Saved: /content/drive/MyDrive/Seg_output/pavan/segment_12.png
Saved: /content/drive/MyDrive/Seg_output/pavan/segment_13.png
Saved: /content/drive/MyDrive/Seg_output/pavan/segment_15.png
Saved unsegmented regions: /content/drive/MyDrive/Seg_output/pavan/segment_0.png


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Assuming predictor, mask_generator, and show_anns are defined elsewhere
def save_mask_objects(image, masks, output_dir, area_threshold):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)


    # Create output directory
    output_dir = "/content/drive/MyDrive/SAM_output/images/"
    os.makedirs(output_dir, exist_ok=True)

    # Initialize an empty array to track used regions
    used_regions = np.zeros(image.shape[:2], dtype=bool)

    # Save each segment as an image with a transparent background
    for idx, mask in enumerate(masks):
        # Extract the segmentation mask
        segmentation_mask = mask['segmentation']
        sorted_masks = sorted(segmentation_mask, key=lambda m: np.count_nonzero(m), reverse=True)
        # Ensure non-overlapping by subtracting already used regions
        unique_mask = np.logical_and(sorted_masks, ~used_regions)

        # Update used regions to include the current mask
        used_regions = np.logical_or(used_regions, unique_mask)

        # Skip if the mask is empty after removing overlaps
        if not np.any(unique_mask):
            continue

        # Create a blank RGBA image (adds alpha channel)
        transparent_image = np.zeros((image.shape[0], image.shape[1], 4), dtype=np.uint8)

        # Copy the segment's RGB values to the new image
        transparent_image[unique_mask, :3] = image[unique_mask]

        # Set alpha channel (255 for the mask region, 0 for background)
        transparent_image[unique_mask, 3] = 255

        # Save the segmented image
        segment_filename = os.path.join(output_dir, f"segment_{idx}.png")
        cv2.imwrite(segment_filename, cv2.cvtColor(transparent_image, cv2.COLOR_RGBA2BGRA))
        print(f"Saved: {segment_filename}")
# Example usage
image_path = "/content/stary_night.jpg"
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
output_dir = "/content/drive/MyDrive/Seg_output"
masks2 = mask_generator_3.generate(image)

# Save non-overlapping masks
save_mask_objects(image, masks2, output_dir, area_threshold=0)


error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


Non-overlapping segments

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
def remove_overlapping_masks(masks):
    """
    Refined method to handle overlapping segmentation masks.

    Args:
    - masks: List of binary masks (numpy arrays).

    Returns:
    - List of adjusted binary masks that minimize overlaps.
    """
    # Sort masks by area (largest to smallest)
    sorted_masks = sorted(masks, key=lambda m: np.count_nonzero(m), reverse=True)
    final_masks = []
    accumulated_mask = np.zeros_like(masks[0], dtype=bool)

    for mask in sorted_masks:
        # Preserve the original mask by reducing overlap only on already accumulated areas
        overlap = np.logical_and(mask, accumulated_mask)
        adjusted_mask = mask.copy()

        if np.any(overlap):
            adjusted_mask[overlap] = 0  # Remove the overlapping portion

        # Add the adjusted mask to the final list if it still contains valid data
        if np.any(adjusted_mask):
            final_masks.append(adjusted_mask)
            # Update accumulated mask to include the new adjusted mask
            accumulated_mask = np.logical_or(accumulated_mask, adjusted_mask)

    return final_masks

# def remove_overlapping_masks(masks):
#     """
#     Remove overlaps between segmentation masks.

#     Args:
#     - masks: List of binary masks (numpy arrays).

#     Returns:
#     - List of non-overlapping binary masks.
#     """
#     final_masks = []
#     accumulated_mask = np.zeros_like(masks[0], dtype=bool)

#     for mask in masks:
#         # Remove overlapping regions by subtracting accumulated mask
#         non_overlapping_mask = np.logical_and(mask, ~accumulated_mask)
#         if np.any(non_overlapping_mask):
#             final_masks.append(non_overlapping_mask)
#             # Update accumulated mask
#             accumulated_mask = np.logical_or(accumulated_mask, non_overlapping_mask)

#     return final_masks

def save_mask_objects(image, masks, output_dir, area_threshold):
    """
    Save non-overlapping segmentation masks as images with transparent backgrounds.

    Args:
    - image: Original RGB image (numpy array).
    - masks: List of masks (dictionary with 'segmentation' key).
    - output_dir: Directory to save the segmented mask images.
    """
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    binary_masks = [mask['segmentation'] for mask in masks]
    non_overlapping_masks = remove_overlapping_masks(binary_masks)
    #Filter and sort masks based on area (largest to smallest)
    sorted_masks = sorted(non_overlapping_masks, key=lambda m: np.count_nonzero(m), reverse=True)

    # Discard masks smaller than the area threshold
    sorted_masks = [mask for mask in sorted_masks if np.count_nonzero(mask) >= area_threshold]

    #for i, mask in enumerate(non_overlapping_masks):#sorted_masks):

    #print(non_overlapping_masks)
    for i, mask in enumerate(sorted_masks):
        # Create an empty RGBA image
        rgba_image = np.zeros((image.shape[0], image.shape[1], 4), dtype=np.uint8)

        # Apply segmentation mask to the alpha channel
        rgba_image[:, :, 3] = mask.astype(np.uint8) * 255

        # Apply the original RGB image to the RGB channels
        rgba_image[:, :, :3] = image

        # Save segmented region with transparent background
        output_path = os.path.join(output_dir, f"segmented_region_{i}.png")
        cv2.imwrite(output_path, rgba_image)

# Example usage
image_path = "/content/pavan.jpg"
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
output_dir = "/content/drive/MyDrive/Seg_output"
masks2 = mask_generator_3.generate(image)

# Save non-overlapping masks
save_mask_objects(image, masks2, output_dir, area_threshold=0)


Saved: /content/drive/MyDrive/SlimSAM_colab/images/segment_0.png
Saved: /content/drive/MyDrive/SlimSAM_colab/images/segment_1.png
Saved: /content/drive/MyDrive/SlimSAM_colab/images/segment_2.png
Saved: /content/drive/MyDrive/SlimSAM_colab/images/segment_3.png
Saved: /content/drive/MyDrive/SlimSAM_colab/images/segment_5.png
Saved: /content/drive/MyDrive/SlimSAM_colab/images/segment_6.png
Saved: /content/drive/MyDrive/SlimSAM_colab/images/segment_7.png
Saved: /content/drive/MyDrive/SlimSAM_colab/images/segment_9.png


sort sam segments with area (default sam)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

def save_mask_objects_sorted_by_area(image, masks, output_dir, area_threshold=5000):
    # Calculate areas and filter masks based on the area threshold
    masks_with_area = []
    for mask in masks:
        area = np.sum(mask['segmentation'])
        masks_with_area.append({'mask': mask, 'area': area})
        print(f"Segment Area: {area}")  # Debug: Print area of each segment
    # masks_with_area = [
    #     {'mask': mask, 'area': np.sum(mask['segmentation'])} for mask in masks
    # ]
    filtered_masks = [m for m in masks_with_area if m['area'] >= area_threshold]

    # Sort the filtered masks based on area in descending order
    sorted_masks = sorted(filtered_masks, key=lambda x: x['area'], reverse=True)

    for i, mask_data in enumerate(sorted_masks):
        segmentation = mask_data['mask']['segmentation']

        # Create an empty RGBA image
        rgba_image = np.zeros((image.shape[0], image.shape[1], 4), dtype=np.uint8)

        # Apply segmentation mask to the alpha channel
        rgba_image[:, :, 3] = segmentation.astype(np.uint8) * 255

        # Apply the original RGB image to the RGB channels
        rgba_image[:, :, :3] = image

        # Save segmented region with transparent background
        cv2.imwrite(f"{output_dir}/segmented_region_{i}.png", rgba_image)

# Example usage
image_path = "/content/pavan.jpg"
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
output_dir = "/content/drive/MyDrive/Seg_output"
masks2 = mask_generator_3.generate(image)

# Set area threshold to 5000
save_mask_objects_sorted_by_area(image, masks2, output_dir, area_threshold=50000)


Segment Area: 14718
Segment Area: 1583005
Segment Area: 1921543
Segment Area: 2632025
Segment Area: 366686
Segment Area: 682982
Segment Area: 3096096
Segment Area: 359280
Segment Area: 267185
Segment Area: 86288
Segment Area: 1173839
Segment Area: 642335
Segment Area: 141989
Segment Area: 635274
